## Import libraries

In [1]:
# standard setup
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import cv2
import random
import time

# useful helper function
from helpers import imshow

## Adding Dense Optical Flow on Video Frame

In [2]:
# parameters for farneback optical flow
fb_params = dict( pyr_scale = 0.5, 
                  levels = 3, 
                  winsize = 5, 
                  iterations = 3, 
                  poly_n = 5,
                  poly_sigma = 1.2, 
                  flags = 0 )

In [3]:
def initializeCamera(w):
    '''
    @w: width of the video frame
    @returns: camera object
    '''
    camera = cv2.VideoCapture(0)
    camera.set(cv2.CAP_PROP_FRAME_WIDTH, w) 
    camera.set(cv2.CAP_PROP_FRAME_HEIGHT, w * 3/4) 
    camera.set(cv2.CAP_PROP_EXPOSURE,-4) 
    return camera

In [4]:
def draw_motion_field(u, v, scale, stride, color=(0, 255, 0)):
    """Draws motion fields given the displacements in X and Y directions.
    
    """
    img_out = np.zeros((v.shape[0], u.shape[1], 3), dtype=np.uint8)

    for y in range(0,v.shape[0],stride):
        for x in range(0,u.shape[1],stride):
            cv2.line(img_out, (x, y), (x + int(u[y, x] * scale), y + int(v[y, x] * scale)), color, 1)
            cv2.circle(img_out, (x + int(u[y, x] * scale), y + int(v[y, x] * scale)), 1, color, 1)
            
    return img_out

In [5]:
def draw_motion_field_on_frame(frame, u, v, scale, stride, color=(0, 0, 0)):
    """Draws motion fields given the displacements in X and Y directions.
    
    """
    img_out = frame.copy()

    for y in range(5,v.shape[0],stride):
        for x in range(5,u.shape[1],stride):
            cv2.line(img_out, (x, y), (x + int(u[y, x] * scale), y + int(v[y, x] * scale)), color, 2)
            cv2.circle(img_out, (x + int(u[y, x] * scale), y + int(v[y, x] * scale)), 3, color, 3)
            
    return img_out

In [14]:
from IPython.display import clear_output

# Initialize camera
camera = initializeCamera(640)

# Flip frame
ret, prev_frame = camera.read()
prev_frame = cv2.flip(prev_frame, 1)
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

while True:
    clear_output(True)

    # Get current frame at flip it
    ret, next_frame = camera.read()
    next_frame = cv2.flip(next_frame, 1)
    next_gray = cv2.cvtColor(next_frame, cv2.COLOR_BGR2GRAY)

    # Calculate the dense optical flow
    flow = cv2.calcOpticalFlowFarneback(prev_gray, next_gray, None, **fb_params)
    
    # Obtain the flow magnitude and direction angle
    mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])
    
    # Add vector field to a video frame
    field_img = draw_motion_field_on_frame(next_frame, flow[...,0], flow[...,1], scale=2, stride=50)
        
    cv2.imshow('Frame', next_frame)
    cv2.imshow('Field', field_img)
    
    prev_gray = next_gray
    
    if cv2.waitKey(5) == 27:
        break      
        
cv2.destroyAllWindows()
camera.release()
cv2.waitKey(1) # extra waitKey sometimes needed to close camera window

-1

## Find General Direction and Magnitue

In [15]:
# Helper function to find magnitude of the vector
def get_mag(dx, dy):
    return np.linalg.norm([dx, dy])


# Helper function to find angle of the vector
def get_angle(dx, dy):
    x_direction = [1, 0]
    v = [dx,dy] / np.linalg.norm([dx, dy])
    cos_theta = np.dot(v, x_direction) 
    orientation = np.cross(x_direction, v)
    if orientation >= 0:
        angle = np.arccos(cos_theta)*(180/np.pi)
    else:
        angle = 360 - np.arccos(cos_theta)*(180/np.pi)
    return angle

In [97]:
def average_displacement(frame, u, v, scale, stride, color=(0, 0, 0)):
    img_out = frame.copy()

    mag_ave = 0
    ang_ave = 0
    
    # angles[0] = 0
    # angles[1] = 90
    # angles[2] = 180
    # angles[3] = 270
    angles = np.zeros(4)
    magnitudes = np.zeros(4)
    
    directions = ["RIGHT", "DOWN", "LEFT", "UP"]
    
    for y in range(0,u.shape[0],stride):
        for x in range(0,u.shape[1],stride):
            dx, dy = u[y, x], v[y, x]
            mag = get_mag(dx, dy)
            ang = get_angle(dx, dy)
              
            if mag > 30:
                if ang>=315 or ang<45:
                    angles[0] += 1
                    magnitudes[0] += 1
                elif 45 <= ang < 135:
                    angles[1] += 1
                    magnitudes[1] += 1
                elif 135 <= ang < 225:
                    angles[2] += 1
                    magnitudes[2] += 1
                elif 225 <= ang < 315:
                    angles[3] += 1
                    magnitudes[3] += 1                
            cv2.line(img_out, (x, y), (x + int(dx * scale), y + int(dy * scale)), color, 2)
            cv2.circle(img_out, (x + int(dx * scale), y + int(dy * scale)), 3, color, 3) 
                

    if np.sum(angles) >= 0:
        if np.max(angles) >= 10:
            print np.max(angles)
            print np.argmax(angles)
            print directions[np.argmax(angles)]

    return img_out
    

In [104]:
### from IPython.display import clear_output

# Initialize camera
camera = initializeCamera(640)

# Flip frame
ret, prev_frame = camera.read()
prev_frame = cv2.flip(prev_frame, 1)
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

while True:
    clear_output(True)

    # Get current frame at flip it
    ret, next_frame = camera.read()
    next_frame = cv2.flip(next_frame, 1)
    next_gray = cv2.cvtColor(next_frame, cv2.COLOR_BGR2GRAY)

    # Calculate the dense optical flow
    flow = cv2.calcOpticalFlowFarneback(prev_gray, next_gray, None, **fb_params)
    
    # Obtain the flow magnitude and direction angle
    mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])
    
#     print mag.shape
#     print "Magnitude", mag[240,320]
#     print "Direction", ang[240,320]*180%360
#     print "Direction", ang[200,300]*180%360
#     print "Direction", ang[100,200]*180%360
    
    
    # Add vector field to a video frame
    field_img = average_displacement(next_frame, flow[...,0], flow[...,1], scale=2, stride=30)
    
    cv2.imshow('Frame', next_frame)
    cv2.imshow('Field', field_img)
    
    prev_gray = next_gray
    
    if cv2.waitKey(5) == 27:
        break      
        
cv2.destroyAllWindows()
camera.release()
cv2.waitKey(1) # extra waitKey sometimes needed to close camera window

-1

## Find General Direction and Magnitue

In [15]:
# Helper function to find magnitude of the vector
def get_mag(dx, dy):
    return np.linalg.norm([dx, dy])


# Helper function to find angle of the vector
def get_angle(dx, dy):
    x_direction = [1, 0]
    v = [dx,dy] / np.linalg.norm([dx, dy])
    cos_theta = np.dot(v, x_direction) 
    orientation = np.cross(x_direction, v)
    if orientation >= 0:
        angle = np.arccos(cos_theta)*(180/np.pi)
    else:
        angle = 360 - np.arccos(cos_theta)*(180/np.pi)
    return angle

In [97]:
def average_displacement(frame, u, v, scale, stride, color=(0, 0, 0)):
    img_out = frame.copy()

    mag_ave = 0
    ang_ave = 0
    
    # angles[0] = 0
    # angles[1] = 90
    # angles[2] = 180
    # angles[3] = 270
    angles = np.zeros(4)
    magnitudes = np.zeros(4)
    
    directions = ["RIGHT", "DOWN", "LEFT", "UP"]
    
    for y in range(0,u.shape[0],stride):
        for x in range(0,u.shape[1],stride):
            dx, dy = u[y, x], v[y, x]
            mag = get_mag(dx, dy)
            ang = get_angle(dx, dy)
              
            if mag > 30:
                if ang>=315 or ang<45:
                    angles[0] += 1
                    magnitudes[0] += 1
                elif 45 <= ang < 135:
                    angles[1] += 1
                    magnitudes[1] += 1
                elif 135 <= ang < 225:
                    angles[2] += 1
                    magnitudes[2] += 1
                elif 225 <= ang < 315:
                    angles[3] += 1
                    magnitudes[3] += 1                
            cv2.line(img_out, (x, y), (x + int(dx * scale), y + int(dy * scale)), color, 2)
            cv2.circle(img_out, (x + int(dx * scale), y + int(dy * scale)), 3, color, 3) 
                

    if np.sum(angles) >= 0:
        if np.max(angles) >= 10:
            print np.max(angles)
            print np.argmax(angles)
            print directions[np.argmax(angles)]

    return img_out
    

In [104]:
### from IPython.display import clear_output

# Initialize camera
camera = initializeCamera(640)

# Flip frame
ret, prev_frame = camera.read()
prev_frame = cv2.flip(prev_frame, 1)
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

while True:
    clear_output(True)

    # Get current frame at flip it
    ret, next_frame = camera.read()
    next_frame = cv2.flip(next_frame, 1)
    next_gray = cv2.cvtColor(next_frame, cv2.COLOR_BGR2GRAY)

    # Calculate the dense optical flow
    flow = cv2.calcOpticalFlowFarneback(prev_gray, next_gray, None, **fb_params)
    
    # Obtain the flow magnitude and direction angle
    mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])
    
#     print mag.shape
#     print "Magnitude", mag[240,320]
#     print "Direction", ang[240,320]*180%360
#     print "Direction", ang[200,300]*180%360
#     print "Direction", ang[100,200]*180%360
    
    
    # Add vector field to a video frame
    field_img = average_displacement(next_frame, flow[...,0], flow[...,1], scale=2, stride=30)
    
    cv2.imshow('Frame', next_frame)
    cv2.imshow('Field', field_img)
    
    prev_gray = next_gray
    
    if cv2.waitKey(5) == 27:
        break      
        
cv2.destroyAllWindows()
camera.release()
cv2.waitKey(1) # extra waitKey sometimes needed to close camera window

-1